Used resources:
- Lab 6 and 7
- Sharmal, P. (2019) Simple multi layer neural network implementation. <https://intellipaat.com/community/9507/simple-multi-layer-neural-network-implementation>
- https://towardsdatascience.com/coding-a-2-layer-neural-network-from-scratch-in-python-4dd022d19fd2

In [17]:
import os
import pandas as pd
import numpy as np
import random
from scipy.stats import truncnorm
from torchvision.datasets import MNIST
import time

In [27]:
#class Neural Networks for initiating an NN
class NN():
    def __init__ (self, layers, epochs, learning_rate):
        self.layers = layers
        self.epochs = epochs
        self.learning_rate = learning_rate
    
        self.params = self.intro()
    def sigmoid(self, x, derivative=False):
        if derivative:
            return (np.exp(-x))/((np.exp(-x)+1)**2)
        return 1/(1 + np.exp(-x))
    def relu (self, x, derivative=False):
        if derivative:
            return max(0,x)
        else:    
            if x<0:
                return 0
            if x>0:
                return 1

    def softmax(self, x, derivative=False):
        # Numerically stable with large exponentials
        exps = np.exp(x - x.max())
        if derivative:
            return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))
        return exps / np.sum(exps, axis=0)
    #loss method - cross entropy
    def cross_entropy(output, target):
        return -np.mean(target*np.log(output))

 
    def intro (self):
        # number of nodes in each layer
        params  = {}
        
        for i in range(len(self.layers)-1):
            key = ("w" + str(i+1))
            value = np.random.randn(self.layers[i+1], self.layers[i]) * np.sqrt(1. / self.layers[i+1])                
            params [key] = value
        return params 
    
    def forward_pass(self, x_train):
        params = self.params
 
        # input layer activations becomes sample
        params['a0'] = x_train
                
        for i in range(len(self.layers)-1):
            #for output layer
            if i == (len(self.layers)-1):
                key = "z" +str(len(self.layers)-1)
                value = np.dot(params["w" + str(len(self.layers)-1)], params["a" + str(len(self.layers)-1)])
                params[key] = value

                key = "a" + str(i+1)
                value = self.softmax(params["z" + str(len(self.layers)-1)])
                params[key] = value
                
            else:
                key = "z" +str(i+1)
                value = np.dot(params["w" + str(i+1)], params["a" + str(i)])
                params[key] = value

                key = "a" + str(i+1)
                value = self.sigmoid(params["z" + str(i+1)])
                params[key] = value
        
        return params['a' + str(len(self.layers)-1)]


    def backward_pass(self, y_train, output):
        params = self.params
        
        change_weights = {}
        
        
        #for output layer
        error = 2 * (output - y_train) / output.shape[0] * self.softmax(params["z" + str(len(self.layers)-1)], derivative=True)
        change_weights["w" + str(len(self.layers)-1)] = np.outer(error, params["a" + str(len(self.layers)-2)])

        for i in reversed(range(len(self.layers)-2)):
            error = np.dot(params["w" + str(i+1)].T, error) * self.sigmoid(params["z" + str(i)], derivative=True)
            change_weights["w" + str(i)] = np.outer(error, params["a" + str(i-1)])

        return change_weights
    
    def update_network_parameters(self, changes_to_weights):

        for key, value in changes_to_weights.items():
            self.weights[key] -= self.l_rate * value
            self.a[key] -= self.l_rate * value
            self.z[key] -= self.l_rate * value

            
    def accuracy(self, x_test, y_test):
        predictions = []
        for x, y in zip(x_test, y_test):
            output = self.forward_pass(x)
            pred = np.argmax(output)
            predictions.append(pred == np.argmax(y))
        
        return np.mean(predictions)

    def train(self, x_train, y_train):
        start_time = time.time()
        for iteration in range(self.epochs):
            for x, y in zip(x_train, y_train):
                output = self.forward_pass(x)
                changes_to_weights = self.backward_pass(y_train, output)
                self.update_network_parameters(changes_to_weights)

            accuracy = self.accuracy(x_val, y_val)
            print('Epoch: {0}, Time Spent: {1:.2f}s, Accuracy: {2:.2f}%'.format(
                iteration+1, time.time() - start_time, accuracy * 100
            ))        
    